<a href="https://colab.research.google.com/github/MagaliDrumare/SQLite/blob/main/SQLlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQLite

>[SQLite](https://en.wikipedia.org/wiki/SQLite) is a database engine written in the C programming language. It is not a standalone app; rather, it is a library that software developers embed in their apps. As such, it belongs to the family of embedded databases. It is the most widely deployed database engine, as it is used by several of the top web browsers, operating systems, mobile phones, and other embedded systems.

In this walkthrough we'll create a simple conversation chain which uses `ConversationEntityMemory` backed by a `SqliteEntityStore`.

In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00


## Usage

To use the storage you need to provide only 2 things:

1. Session Id - a unique identifier of the session, like user name, email, chat id etc.
2. Connection string - a string that specifies the database connection. For SQLite, that string is `slqlite:///` followed by the name of the database file.  If that file doesn't exist, it will be created.

In [ ]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

chat_message_history = SQLChatMessageHistory(
    session_id="test_session_id", connection_string="sqlite:///sqlite.db"
)

chat_message_history.add_user_message("Hello")
chat_message_history.add_ai_message("Hi")

In [ ]:
chat_message_history.messages

[HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi', additional_kwargs={}, response_metadata={})]

## Chaining

We can easily combine this message history class with [LCEL Runnables](/docs/how_to/message_history)

To do this we will want to use OpenAI, so we need to install that.  We will also need to set the OPENAI_API_KEY environment variable to your OpenAI key.

```bash
pip install -U langchain-openai

export OPENAI_API_KEY='sk-xxxxxxx'
```

In [ ]:
pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 1.8 MB/s eta 0:00:00


In [ ]:
import os
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

In [ ]:
os.environ["OPENAI_API_KEY"] = 'sk-proj-XXXXXXX.......'

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = prompt | ChatOpenAI()

In [ ]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: SQLChatMessageHistory(
        session_id=session_id, connection_string="sqlite:///sqlite.db"
    ),
    input_messages_key="question",
    history_messages_key="history",
)

In [ ]:
# This is where we configure the session id
config = {"configurable": {"session_id": "<SQL_SESSION_ID>"}}

In [ ]:
chain_with_history.invoke({"question": "Hi! I'm Bob, I'am living in Paris"}, config=config)

AIMessage(content="Hello, Bob! It's nice to meet you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 28, 'total_tokens': 46, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bf4pFd4luoBy6ThLtssu97WQaQX9W', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--521e8cd2-0d33-4d46-a81d-f48ae5a8d269-0', usage_metadata={'input_tokens': 28, 'output_tokens': 18, 'total_tokens': 46, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
chain_with_history.invoke({"question": "Whats my name and where I am living?"}, config=config)

AIMessage(content='Your name is Bob, and you mentioned that you are living in Paris. How can I help you further?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 63, 'total_tokens': 85, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bf4pIQkn565UTHzahTtAxUE2WmGam', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--545e1a18-13e4-4ff9-8ce8-b1b4b27fdbcd-0', usage_metadata={'input_tokens': 63, 'output_tokens': 22, 'total_tokens': 85, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
chain_with_history.invoke({"question": "I love cooking books"}, config=config)

AIMessage(content="That's great to hear, Bob! Cooking books can be a fantastic source of inspiration and new recipes. Is there a specific type of cuisine you enjoy cooking the most, or are you looking for recommendations on cooking books?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 97, 'total_tokens': 141, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bf4pKCUAXKubu38eK8c13dpwjKJug', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--56fec0cf-e8ad-4180-8038-3da28db17d12-0', usage_metadata={'input_tokens': 97, 'output_tokens': 44, 'total_tokens': 141, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning':

In [ ]:
chain_with_history.invoke({"question": "Where I am living?"}, config=config)

AIMessage(content='You mentioned earlier that you are living in Paris. Is there anything specific you would like to know or discuss related to Paris or cooking books?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 154, 'total_tokens': 182, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bf4pMWUAbsvRkeEVGXQkvFr2pFWOR', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e3b4f5c8-b742-45c6-98aa-d25a2e94eec4-0', usage_metadata={'input_tokens': 154, 'output_tokens': 28, 'total_tokens': 182, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
chain_with_history.invoke({"question": "What do I like?"}, config=config)

AIMessage(content='You mentioned that you love cooking books, Bob. Is there anything specific related to cooking or culinary arts that you would like to discuss or know more about? Feel free to share your interests or ask any questions you may have.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 281, 'total_tokens': 326, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BegMJjgDEV56jKecVROagmwvhp2oe', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--7e1daf7d-5520-4d78-b004-4aa31f1b79ae-0', usage_metadata={'input_tokens': 281, 'output_tokens': 45, 'total_tokens': 326, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0,